In [2]:
from os import fsencode, listdir, fsdecode
from PIL import Image, ImageDraw

directory = fsencode('input')
for file in listdir(directory):
    filename = fsdecode(file)
    if filename.endswith('jpg'):
        # print('Filename {}'.format(filename))
        img = Image.open('input/' + filename)
        # img.show()
        # With this set we can crop about 10% from bottom to delete rulers
        crop_mult_y = 0.9
        w, h = img.size
        img = img.crop((0, 0, w, h * crop_mult_y))
        w, h = img.size
        # img.show()
        # Sample 10 pixels from where there should always be solid background
        samples = 0
        for i in range(1000, 1010):
            samples += sum(img.getpixel((i, 100))) / 3
        sample_average = samples / 10
        # print('Sample average is {}'.format(sample_average))
        center_x = int(w / 2)
        center_y = int(h / 2)
        
        upper_edge_y = None
        for i in range(0, h):
            average_pixel_rgb = sum(img.getpixel((center_x, i))) / 3
            # print (average_pixel_rgb)
            if not (sample_average * 0.95 < average_pixel_rgb < sample_average * 1.05):
                # We have likely hit the upper edge of the real image
                draw = ImageDraw.Draw(img)
                draw.line((0, i, w, i), fill=128)
                # img.show()
                upper_edge_y = i
                break
    
        lower_edge_y = None
        for i in range(h - 1, 0, -1):
            average_pixel_rgb = sum(img.getpixel((center_x, i))) / 3
            # print (average_pixel_rgb)
            if not (sample_average * 0.95 < average_pixel_rgb < sample_average * 1.05):
                # We have likely hit the lower edge of the real image
                draw = ImageDraw.Draw(img)
                draw.line((0, i, w, i), fill=128)
                # img.show()
                lower_edge_y = i
                break
    
        right_edge_x = None
        for i in range(w - 1, 0, -1):
            average_pixel_rgb = sum(img.getpixel((i, center_y))) / 3
            # print (average_pixel_rgb)
            if not (sample_average * 0.95 < average_pixel_rgb < sample_average * 1.05):
                # We have likely hit the right edge of the real image
                draw = ImageDraw.Draw(img)
                draw.line((i, 0, i, h), fill=128)
                # img.show()
                right_edge_x = i
                break
                
        left_edge_x = None
        left_edge_start_looking_x = None
        # Wait until at least 30 pixels of nice gray
        gray_detected = 0
        j = 0
        while gray_detected < 30:
            average_pixel_rgb = sum(img.getpixel((j, center_y))) / 3
            if sample_average * 0.95 < average_pixel_rgb < sample_average * 1.05:
                gray_detected += 1
            else:
                gray_detected = 0
            j += 1
        for i in range(j, w):
            average_pixel_rgb = sum(img.getpixel((i, center_y))) / 3
            # print (average_pixel_rgb)
            if not (sample_average * 0.95 < average_pixel_rgb < sample_average * 1.05):
                # We have likely hit the left edge of the real image
                draw = ImageDraw.Draw(img)
                draw.line((i, 0, i, h), fill=128)
                # img.show()
                left_edge_x = i
                break
    
        # img.show()
        with open('output/' + filename, 'w') as f:
            img.save(f)



IndexError: image index out of range